In [1]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder

class pre:
    def creation_data(self, path_csv):
        df = pd.read_csv(path_csv)
        return df
    
    def declaration_var(self, data, target):
        
        target_1 = data[target]
        features = data.drop(target, axis=1)
        names_f = {name:set() for name in features}
        for i in features:
            index_drop = features[i][features[i].isnull() == True].index
            features.drop(index_drop, inplace = True)    
            target_1.drop(index_drop, inplace = True)
        
        for i in features:
            for j in features[i]:
                names_f[i].add(j)   
                    
        index_targ = target_1[target_1.isnull() == True].index
        target_1.drop(index_targ, inplace=True)   
        features.drop(index_targ, inplace=True)                    
        return target_1, features, names_f
    
    def one_hot(self, variable, features):
        # for category in variable:
        #     if category not in features.columns:
        #         features[category] = 0  # Agrega la categoría con valores 0 si no está presente
        encoder = OneHotEncoder()
        feature_encoded = encoder.fit_transform(features[variable])
        df_one_hot = pd.DataFrame(feature_encoded.toarray(), columns=encoder.get_feature_names_out(variable))
        features.drop(variable, axis=1, inplace=True)
        names_one = [name for name in df_one_hot]
        df_one_hot[names_one] = df_one_hot[names_one].astype('int8')
        features.reset_index(drop=True, inplace=True)
        df_one_hot.reset_index(drop=True, inplace=True)
        features = pd.concat([features, df_one_hot], axis=1)
        return features 

    def replace(self, variable, var_1, var_2, features):
        new_values = [var_1 if value == var_1 or value == var_2 else value for value in features[variable]]
        features[variable] = new_values
        return features



In [2]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import OneHotEncoder


processing = pre()

def pipe(categorical_features, features):
    num_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    cat_transformer = Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))])
    return num_transformer, cat_transformer

def column_trans(num_transformer, num_features, cat_transformer, cat_features):
    preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', num_transformer, num_features),
            ('categorical', cat_transformer, cat_features)
        ]
    )
    return preprocessor

def gen_pipe( preprocessor):
    pipeline = Pipeline(
        steps = [('preprocessor_column', preprocessor),
                ('linear_model', LinearRegression())]
    )
    return pipeline

In [3]:
from pipeline import pipe, column_trans, gen_pipe
from sklearn.compose import ColumnTransformer
import numpy as np 

data = pre().creation_data('Salary_Data.csv')
target, features, names_f = pre().declaration_var(data, 'Salary')

#Cambiar el tipo de formato
features['Age'] = features['Age'].astype('int16')
features['Years of Experience'] = features['Years of Experience'].astype('int64')

# Unir las variables repetidas
features = pre().replace('Education Level', "Bachelor's", "Bachelor's Degree", features)
features = pre().replace('Education Level', "Master's", "Master's Degree", features)
features = pre().replace('Education Level', "PhD", "phD", features)

num_features = ['Age', 'Years of Experience']
cat_features = ['Gender', 'Education Level', 'Job Title']

#Convertir las variables catégoricas en OneHotEncoder
num_transformer, cat_transformer = pipe(cat_features, features)
preprocessor = column_trans(num_transformer, num_features, cat_transformer, cat_features)

from sklearn.model_selection import train_test_split 
X_train, X_test,y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

linear_model = gen_pipe(preprocessor)

reg_l = linear_model.fit(X_train, y_train)

print(reg_l)

from sklearn.metrics import r2_score, mean_squared_error
prediction_linear = linear_model.predict(X_test)
print('r2',r2_score(y_test, prediction_linear))
print('mse', mean_squared_error(y_test, prediction_linear))
print(np.mean(target))
print('rmse', np.sqrt(mean_squared_error(y_test, prediction_linear)))

Pipeline(steps=[('preprocessor_column',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age',
                                                   'Years of Experience']),
                                                 ('categorical',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Education Level',
                                                   'Job Title'])])),
                ('linear_model', LinearRegression())])
r2 0.8401837858582851
mse 455709421.883616
115329.25306061511
rmse 21347.351636294745
